<a href="https://colab.research.google.com/github/vu-topics-in-big-data-2022/Project-Incident-Team2/blob/master/load_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Install and import necessary libraries**

In [ ]:
!pip install boto3 pandas plotly
import boto3, json
import pandas as pd
import geopandas as gpd
import plotly.graph_objects as go
import pyarrow as pa
import pyarrow.parquet as pq
from google.colab import files
from plotly.subplots import make_subplots

**AWS Credentials and Clients**

In [2]:
AWS_ACCESS_KEY = ""
AWS_SECRET_KEY = ""
AWS_REGION = "us-east-1"
AWS_SESSION_TOKEN = ""

In [3]:
# Please fill your aws credential information here
credentials = {
    'region_name': AWS_REGION,
    'aws_access_key_id': AWS_ACCESS_KEY,
    'aws_secret_access_key': AWS_SECRET_KEY,
    'aws_session_token': AWS_SESSION_TOKEN
}

In [4]:
session = boto3.session.Session(**credentials)
athena = session.client('athena', region_name='us-east-1')
s3 = session.client('s3')

**Query traffic dataset with Athena**

In [15]:
queryID = athena.start_query_execution(
    QueryString = 'SELECT EXTRACT(WEEK FROM measurement_tstamp), AVG(congestion), AVG(speed) FROM traffic WHERE year=2020 GROUP BY EXTRACT(WEEK FROM measurement_tstamp)',
    QueryExecutionContext = {
        'Database': 'trafficdata'
    }
)['QueryExecutionId']

In [18]:
athena.get_query_execution(
    QueryExecutionId=queryID
)
s3.download_file('vandy-bigdata-finalproject', 'results/' + queryID + '.csv', 'results.csv')
df = pd.read_csv('results.csv').sort_values(by=['_col0'])
df.head()

,_col0,_col1,_col2
19,1,0.077418,29.972729
11,2,0.088786,29.615384
30,3,0.090728,29.566499
47,4,0.085956,29.554328
26,5,0.090543,29.609133


**Reading csv results into pandas for visualization**

In [20]:
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=df['_col0'], y=df['_col2'], name="Avg Speed"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=df['_col0'], y=df['_col1'], name="Avg Congestion"),
    secondary_y=True,
)

# Add figure title
fig.update_layout(
    title_text="Average Congestion and Speed by Week in Davidson County in 2020"
)

# Set x-axis title
fig.update_xaxes(title_text="Week in year")

# Set y-axes titles
fig.update_yaxes(title_text="<b>Average Speed</b>", secondary_y=False)
fig.update_yaxes(title_text="<b>Average Congestion</b>", secondary_y=True)
fig.update_xaxes(tick0=1, dtick=1)

fig.show()

**The code below cleans up the merged data**

In [ ]:
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving USA_Tennessee.geojson to USA_Tennessee.geojson
User uploaded file "USA_Tennessee.geojson" with length 176882310 bytes


In [ ]:
roads=gpd.read_file('USA_Tennessee.geojson')

In [ ]:
roads=roads[roads.County=='DAVIDSON']
len(roads)

8641

In [ ]:
roads_filtered = roads[['XDSegID','RoadName', 'XDGroup']]

In [ ]:
roads_filtered

,XDSegID,RoadName,XDGroup
9033,155692243,S 11TH ST,1608519
9045,155711459,BERRY RD,1609139
9051,155716829,STONE BROOK DR,1609328
9055,155725149,STANLEY ST,1609599
9056,155730113,CHURCH ST,1609748
...,...,...,...
94650,1524643823,S 5TH ST,1614080
94789,1524644550,8TH AVE S,3176751
94794,1524644585,I-40 W,1722191
94893,1524645372,MURFREESBORO PIKE,1621379


In [ ]:
parquet_table = pa.Table.from_pandas(roads_filtered)
pq.write_table(parquet_table, "road.parquet")

In [ ]:
s3.upload_file("road.parquet", "bigdata-seg", "data-roads/road.parquet")

In [ ]:
s3_url = 's3://bucket/folder/bucket.parquet.gzip'
df.to_parquet(s3_url, compression='gzip')

In [ ]:
from google.colab import files
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [1]:
df = pd.read_csv('merged.csv')

In [8]:
df.head()

,latitude,longitude,response_time_sec,weekend_or_not,incident_id,dist_to_seg,xdsegid,month,day,window_of_day,...,cvalue,congestion,extreme_congestion,year,segid,xdgroup,precip,temp,vis,wind_spd
hour_of_day,,,,,,,,,,,,,,,,,,,,,
0,36.150694,-86.742630,383.172791,0.455736,16681.342418,3.408485,1.052878e+09,7.316308,208.150813,0.999956,...,87.906277,0.044504,0.036274,2019.107341,1.040902e+09,3.061428e+06,0.279151,13.742216,14.547009,2.647774
1,36.160059,-86.750177,408.468225,0.472827,15351.868349,3.777035,1.073800e+09,6.863075,192.839340,1.000000,...,88.945100,0.044609,0.040582,2019.050313,1.071528e+09,3.055908e+06,0.274053,14.529627,14.430620,2.684056
2,36.145619,-86.743417,449.964158,0.508961,14934.652330,3.901474,1.087685e+09,6.637993,185.168459,1.000000,...,87.885958,0.048853,0.041265,2019.014337,1.081666e+09,3.056526e+06,0.281147,14.318280,14.087814,2.683990
3,36.147511,-86.745735,428.445103,0.525675,14168.617590,4.212644,1.058999e+09,6.887858,194.142929,1.000000,...,88.759001,0.040551,0.033045,2018.895010,1.036972e+09,3.013231e+06,0.273565,13.932429,14.548504,2.731330
4,36.159164,-86.748723,410.020637,0.428066,14733.326209,3.360425,1.141001e+09,6.383992,178.248673,1.000000,...,88.233597,0.042703,0.029755,2018.966097,1.117177e+09,3.090795e+06,0.283567,14.629634,14.539800,2.792550


In [ ]:
df = df.drop(columns=['county', 'month_1', 'hour_of_day_1', 'day_1', 'window_of_day_1', 'day_of_week_1', 'window_id_1'])

In [ ]:
df['weekend_or_not'] = df['weekend_or_not'].fillna(0).astype(int)
df['response_time_sec'] = df['response_time_sec'].fillna(0).astype(int)
df['year'] = df['year'].fillna(0).astype(int)
df['month'] = df['month'].fillna(0).astype(int)
df['xdgroup'] = df['xdgroup'].fillna(0).astype(int)
df['hour_of_day'] = df['hour_of_day'].fillna(0).astype(int)
df['day'] = df['day'].fillna(0).astype(int)
df['window_of_day'] = df['window_of_day'].fillna(0).astype(int)
df['incident_id'] = df['incident_id'].fillna(0).astype(int)
df['segid'] = df['segid'].fillna(0).astype(int)
df['xd_id'] = df['xd_id'].fillna(0).astype(int)
df['xdsegid'] = df['xdsegid'].fillna(0).astype(int)
 
df.head()

,id_original,latitude,longitude,emdcardnumber,time_utc,time_local,response_time_sec,weekend_or_not,geometry,incident_id,...,reference_speed,travel_time_seconds,confidence_score,cvalue,congestion,extreme_congestion,year,segid,roadname,xdgroup
0,ObjectId(5fb5b99395fad039a6275ff8),36.058841,-86.670452,29A1,12:59.0,12:59.0,584,0,POINT (-86.67045205 36.05884112),27295,...,26.0,54.53,30.0,100.0,0.0,0.0,2020,1524516301,HICKORY HOLLOW PKWY,4391092
1,ObjectId(5fb5b99395fad039a6275ff8),36.058841,-86.670452,29A1,12:59.0,12:59.0,584,0,POINT (-86.67045205 36.05884112),27295,...,26.0,51.66,30.0,100.0,0.0,0.0,2020,1524516301,HICKORY HOLLOW PKWY,4391092
2,ObjectId(5fb5b99395fad039a6275ff8),36.058841,-86.670452,29A1,12:59.0,12:59.0,584,0,POINT (-86.67045205 36.05884112),27295,...,26.0,50.47,30.0,100.0,0.0,0.0,2020,1524516301,HICKORY HOLLOW PKWY,4391092
3,ObjectId(5fb5b99395fad039a6275ff8),36.058841,-86.670452,29A1,12:59.0,12:59.0,584,0,POINT (-86.67045205 36.05884112),27295,...,26.0,46.99,30.0,100.0,0.0,0.0,2020,1524516301,HICKORY HOLLOW PKWY,4391092
4,ObjectId(5fb5b99395fad039a6275ff8),36.058841,-86.670452,29A1,12:59.0,12:59.0,584,0,POINT (-86.67045205 36.05884112),27295,...,26.0,45.13,30.0,100.0,0.0,0.0,2020,1524516301,HICKORY HOLLOW PKWY,4391092


In [ ]:
table = pa.Table.from_pandas(df)

In [ ]:
pq.write_table(table, 'everything_merged.parquet')

In [ ]:
files.download('everything_merged.parquet')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>